In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
from collections import Counter
import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df = pd.read_csv('DIPS_Data.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df = df.dropna(how='all')

In [4]:
#assigns values to each row in the column and removes blank rows
data = df.copy()
data = data.fillna({'case_name':'none'})
data['Case'] = 'xxxx'
value = 'Fees:'
case = 'xxxx'
for row in data.itertuples():
    if row[1] not in ['none', 'Fees:', 'Agents:','Lender Groups:', 'Roll ups:', 'Lenders:']:
        case = row[1]
        data.at[row[0], 'Case'] = case
        continue
    if row[1] == 'Fees:':
        value = 'Fees:'
        continue
    if row[1] == 'Agents:':
        value = 'Agents:'
        continue
    if row[1] == 'Lender Groups:':
        value = 'Lender Groups:'
        continue
    if row[1] == 'Roll ups:':
        value = 'Roll ups:'
        continue
    if row[1] == 'Lenders:':
        value = 'Lenders:'
        continue
    data.at[row[0], 'Case'] = case
    data.at[row[0], 'case_name'] = value
data = data[~data.Case.isin(['xxxx'])]
data = data.reset_index(drop=True)
#data = data.set_index('Case')

In [81]:
starting_list = list(data['case_name'])
clean_list = [x for x in starting_list if str(x) != 'nan']
unique_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
all_cases = [x for x in clean_list if str(x) not in ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']]
total_cases = len(all_cases)
#counts the total number of cases present in the dataframe
#observation_count = Counter(clean_list) #dictionary of count of unique deals, Fees, Agents ect.

#combs through the created list and returns the case names that have missing data i.e No Agents, No Lenders ect.
master_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
case_dict = {}
case_index = []
agent_no_lenders = []
no_data = []
lenders_no_agent = []
one_lender_no_agent = []
no_agent = []

for i in range(0, len(clean_list)):
    if clean_list[i] not in master_list:
        fee_num = 0
        agent_num = 0
        name = clean_list[i]
        case_index.append(i)
        dup_case = name in case_dict
        if not dup_case:
            case_dict[name] = {}
            case_dict[name]['Case_Count'] = 1
            dip_num = 'Dip_Number_1'
        if dup_case:
            case_dict[name]['Case_Count'] = case_dict[name]['Case_Count'] + 1
            dip_num = 'Dip_Number_' + str(case_dict[name]['Case_Count'])
        case_dict[name][dip_num] = {}
        case_dict[name][dip_num]['Data'] = {}
        case_dict[name][dip_num]['Data']['Publishing_Name'] = data.loc[i]['publishing_name']
        case_dict[name][dip_num]['Data']['Currency'] = data.loc[i]['currency']
        case_dict[name][dip_num]['Data']['Loan_Amount'] = data.loc[i]['loan_amount']
        case_dict[name][dip_num]['Data']['Maturity'] = data.loc[i]['maturity']
        case_dict[name][dip_num]['Data']['Facility_Type'] = data.loc[i]['facility_type']
        case_dict[name][dip_num]['Data']['Fixed_Float'] = data.loc[i]['fixed/floating']
        case_dict[name][dip_num]['Data']['Floating_Type'] = data.loc[i]['floating_type']
        case_dict[name][dip_num]['Data']['Investigation_Budget'] = data.loc[i]['investigation_budget']
        case_dict[name][dip_num]['Data']['Carve_Out'] = data.loc[i]['carve_out']
    if clean_list[i] == 'Fees:':
        if fee_num == 0:
            case_dict[name][dip_num]['Fees'] = {}
        fee_num = fee_num + 1
        fee_name = 'Fee_'+str(fee_num)
        case_dict[name][dip_num]['Fees'][fee_name] = {}
        case_dict[name][dip_num]['Fees'][fee_name]['Type'] = data.loc[i]['case_status']
        case_dict[name][dip_num]['Fees'][fee_name]['Sub_Type'] = data.loc[i]['petition_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount'] = data.loc[i]['confirmation_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount_Type'] = data.loc[i]['dismissal_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Duration_Type'] = data.loc[i]['closing_date']
    if clean_list[i] == 'Agents:':
        dup_agent = 'Agents' in case_dict[name][dip_num]#returns true if Agents is already there, and False if it is not
        if not dup_agent:
            case_dict[name][dip_num]['Agents'] = {}
            case_dict[name][dip_num]['Agents'][str(data.loc[i]['petition_date'])] = []
        if dup_agent:
            dup_agent_type = str(data.loc[i]['petition_date']) in case_dict[name][dip_num]['Agents']
            if not dup_agent_type:
                case_dict[name][dip_num]['Agents'][str(data.loc[i]['petition_date'])] = []
        case_dict[name][dip_num]['Agents'][str(data.loc[i]['petition_date'])].append(str(data.loc[i]['case_status']))

In [30]:
added_agents = pd.read_csv('Added_Agents.csv')

In [82]:
added_dict = {}
for i in range(0, len(added_agents.index)):
    name = str(added_agents.loc[i]['Case'])
    agent = added_agents.loc[i]['case_status']
    agent_type = added_agents.loc[i]['petition_date']
    dup_name = name in added_dict
    if not dup_name:
        added_dict[name] = 1
    if dup_name:
        added_dict[name] = added_dict[name] + 1
    dip_num = 'Dip_Number_' + str(added_dict[name])
    case_dict[name][dip_num]['Agents'] = {}
    case_dict[name][dip_num]['Agents'][str(agent_type)] = str(agent)

In [92]:
puller_list = []
list(case_dict.keys())
for i in list(case_dict.keys()):
    for j in range(1, case_dict[i]['Case_Count']):
        if 'Agents' in case_dict[i]['Dip_Number_'+str(j)]:
            if 'Admin Agent' in case_dict[i]['Dip_Number_'+str(j)]['Agents']:
                
                
                puller_list.append((i, (case_dict[i]['Dip_Number_'+str(j)]['Data'], case_dict[i]['Dip_Number_'+str(j)]['Agents']['Admin Agent'])))

In [97]:
case_dict

{'In re Fairway Group Holdings Corp.': {'Case_Count': 2,
  'Dip_Number_1': {'Data': {'Publishing_Name': 'DIP Term Loan',
    'Currency': 'USD',
    'Loan_Amount': 55000000.0,
    'Maturity': 3.0,
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 250000.0,
    'Carve_Out': 1000000.0},
   'Fees': {'Fee_1': {'Type': ' Unused Commitment Fee',
     'Sub_Type': nan,
     'Amount': '0.375',
     'Amount_Type': '%',
     'Duration_Type': 'Monthly'},
    'Fee_2': {'Type': 'Other Fee',
     'Sub_Type': 'Duration Fee',
     'Amount': '0.25',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_3': {'Type': 'Commitment Fee',
     'Sub_Type': nan,
     'Amount': '2',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_4': {'Type': 'Arrangement/Origination Fee',
     'Sub_Type': nan,
     'Amount': '0.5',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'}},
   'Agents': {'Admin Agent': [

In [96]:
pprint.pprint(puller_list)

[('In re Fairway Group Holdings Corp.',
  ({'Carve_Out': 1000000.0,
    'Currency': 'USD',
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 250000.0,
    'Loan_Amount': 55000000.0,
    'Maturity': 3.0,
    'Publishing_Name': 'DIP Term Loan'},
   ['Credit Suisse'])),
 ('In re Peabody Energy Corporation',
  ({'Carve_Out': 7500000.0,
    'Currency': 'USD',
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 25000.0,
    'Loan_Amount': 500000000.0,
    'Maturity': 12.0,
    'Publishing_Name': 'DIP Term Loan'},
   ['CitiBank'])),
 ('In re Peabody Energy Corporation',
  ({'Carve_Out': 7500000.0,
    'Currency': 'USD',
    'Facility_Type': 'Other',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 25000.0,
    'Loan_Amount': 200000000.0,
    'Maturity': 12.0,
    'Publishing_Name': 'DIP Bonding Accommodation Fac

    'Facility_Type': 'Revolver',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': nan,
    'Loan_Amount': 247500000.0,
    'Maturity': 7.0,
    'Publishing_Name': 'DIP Revolver'},
   ['Wells Fargo Bank'])),
 ('In re VER Technologies HoldCo LLC',
  ({'Carve_Out': 1000000.0,
    'Currency': 'USD',
    'Facility_Type': 'Revolver',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 100000.0,
    'Loan_Amount': 300000000.0,
    'Maturity': 6.0,
    'Publishing_Name': 'DIP ABL'},
   ['Bank of America'])),
 ('In re Gregory John te Velde (Lost Valley Farm)',
  ({'Carve_Out': nan,
    'Currency': 'USD',
    'Facility_Type': 'Term Loan',
    'Fixed_Float': nan,
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': nan,
    'Loan_Amount': 400000.0,
    'Maturity': nan,
    'Publishing_Name': 'DIP Loan [Pending]'},
   'Rabobank')),
 ('In re The Rockport Company, LLC',
  ({'Carve_Out': 350000.0,
    'Currency': 'USD',

In [ ]:
for i in range(0, len(puller_list)):
    